# Sneaker Model and Size stock counter (adidas)

In [ ]:
import time
import requests
from discord_webhook import DiscordWebhook
import json

SKU = str(input("SKU: "))
region = input("Region: ").lower()

url = "https://www.adidas.com." + region + "/api/products/"+ SKU + "/availability"
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36','Accept-Encoding': 'gzip, deflate, br','Accept-Language': 'en-US,en;q=0.9,hi;q=0.8'}
dc_url = "https://discordapp.com/api/webhooks/748573860265459722/3Z8Tgd-XRZthdMxPyfS_STJpDzT4dvUOS6itLlDnZ7oYLpXXD_y8YGsVSntV_8IVf7j4"

#BOT username
botname = str(region).upper() + " Stock Check BOT"

#finding country flag for title
if region == "sg":
    flag = ":flag_sg:"
elif region == "my":
    flag = ":flag_my:"
elif region == "au":
    flag = ":flag_au:"
elif region == "gb":
    flag = ":flag_gb:"

start_time = time.time()
duration = int(99999)
run_count = 1
count = 0

#success is 0 or 1 based on whether json_first was picked up when the program was first run
success = 0
error_count = 0

#check_duration in seconds is the frequency you wanna see the updates e.g. 300s = 5 minutes per update
check_duration = 300

#First scrape to set basis of comparison for subsequent scrapes
main_json = requests.get(url, headers = headers)

#if the page was starting from a blocked page
#try:
json_first = json.loads(main_json.text)
status = json_first['availability_status']
    #success += 1

#If the page is blocked it will go through here
#except ValueError:
#    print('Page is being blocked when program run')

start_time = time.time()
print_count = 0

while True:
    try:
        current_time = time.time()
        elapsed_time = current_time - start_time

        if elapsed_time < duration:
            if elapsed_time < (run_count * check_duration):
                main_str = requests.get(url,headers = headers)
                json_dict = json.loads(main_str.text)

                data = {}
                data["embeds"] = []
                embed = {}

                #because the program ran when the page was being blocked, nothing was stored in json_first
                if success == 0:
                    json_first = json_dict
                    status = json_first['availability_status']
                    success += 1

                if json_dict == json_first:
                    #if initial status in 'PREVIEW'
                    if status == 'PREVIEW':
                        if print_count == 0:
                            data["username"] = botname
                            embed["title"] = flag + " " + SKU + " Stock Check - Still in Preview"
                            embed["description"] = 'Awaiting Changes in Stock Count'
                            data["embeds"].append(embed)

                            requests.post(dc_url, data=json.dumps(data), headers={"Content-Type": "application/json"})
                            print_count += 1

                    #if initial status in 'IN_STOCK'
                    elif status == 'IN_STOCK':
                        #Each varation_list item is one size so the len will give the number of sizes available -> each size will be checked for the numbers
                        size_run = len(json_dict['variation_list'])
                        all_sizes = json_dict['variation_list']
                        count = 0
                        output = ""

                        while count < size_run:
                            if (all_sizes[count])['availability'] != 0:
                                size_dict = all_sizes[count]
                                size = size_dict['size']
                                stock = size_dict['availability']
                                output = output + str(size) + ": " + str(stock) + '\n'
                                count += 1

                            #If not available, just move on to the next size and check availability
                            else:
                                count += 1

                        #send webhook once only
                        if (print_count == 0):
                            data["username"] = botname
                            data["description"] = "@here"
                            embed["title"] = flag + " " + SKU + " Stock Check Reported"
                            embed["description"] = output
                            data["embeds"].append(embed)

                            requests.post(dc_url, data=json.dumps(data), headers={"Content-Type": "application/json"})
                            print_count += 1

                    #if initial status in 'NOT_AVAILABLE'
                    elif status == 'NOT_AVAILABLE':
                        if print_count == 0:
                            data["username"] = botname
                            embed["title"] = flag + " " + SKU + " Stock Check - No sizes available"
                            embed["description"] = 'Awaiting Changes in Stock Count'
                            data["embeds"].append(embed)

                            requests.post(dc_url, data=json.dumps(data), headers={"Content-Type": "application/json"})
                            print_count += 1

                #Latest scraped data is different from the existing data
                else:
                    sta_new = json_dict['availability_status']
                    #Initial status was 'PREVIEW'
                    if status == 'PREVIEW':
                        #'PREVIEW' to 'PREVIEW' but something in the API changed -> not sure if this happens but can monitor what changes
                        if sta_new == 'PREVIEW':
                            data["username"] = botname
                            embed["title"] = flag + " " + SKU + " Stock Check Changes Detected"
                            embed["description"] = json_dict
                            data["embeds"].append(embed)

                            requests.post(dc_url, data=json.dumps(data), headers={"Content-Type": "application/json"})

                            #The latest scrape take over the previous statuses etc
                            json_first = json_dict
                            status = sta_new

                        #'PREVIEW' to 'IN_STOCK' -> send webhook notif @here + SIZE RUN uploaded
                        elif sta_new == 'IN_STOCK':
                            size_run = len(json_dict['variation_list'])
                            all_sizes = json_dict['variation_list']
                            count = 0
                            output = ""

                            while count < size_run:
                                if (all_sizes[count])['availability'] != 0:
                                    size_dict = all_sizes[count]
                                    size = size_dict['size']
                                    stock = size_dict['availability']
                                    output = output + str(size) + ": " + str(stock) + '\n'
                                    count += 1

                                #If not available, just move on to the next size and check availability
                                else:
                                    count += 1

                            data["username"] = botname
                            data["description"] = "@here"
                            embed["title"] = flag + " " + SKU + " STOCK MADE AVAILABLE!!!"
                            embed["description"] = output
                            data["embeds"].append(embed)

                            requests.post(dc_url, data=json.dumps(data), headers={"Content-Type": "application/json"})

                            #The latest scrape take over the previous statuses etc
                            json_first = json_dict
                            status = sta_new

                        #'PREVIEW' to 'NOT_AVAILABLE'
                        elif sta_new == "NOT_AVAILABLE":
                            data["username"] = botname
                            embed["title"] = flag + " " + SKU + " Stock unavailable"
                            embed["description"] = "The availability_status went from PREVIEW to NOT_AVAILABLE..."
                            data["embeds"].append(embed)

                            requests.post(dc_url, data=json.dumps(data), headers={"Content-Type": "application/json"})

                            #The latest scrape take over the previous statuses etc
                            json_first = json_dict
                            status = sta_new

                    #Initial status was 'IN_STOCK'
                    elif status == "IN_STOCK":
                        sta_new = json_dict['availability_status']
                        #'IN_STOCK' -> 'PREVIEW'
                        if sta_new == 'PREVIEW':
                            data["username"] = botname
                            embed["title"] = flag + " " + SKU + " change detected"
                            embed["description"] = "No more Stock..."
                            data["embeds"].append(embed)

                            requests.post(dc_url, data=json.dumps(data), headers={"Content-Type": "application/json"})

                        #'IN_STOCK' -> 'IN_STOCK'
                        elif sta_new == 'IN_STOCK':
                            size_run = len(json_dict['variation_list'])
                            all_sizes = json_dict['variation_list']
                            count = 0
                            output = ""

                            while count < size_run:
                                if (all_sizes[count])['availability'] != 0:
                                    size_dict = all_sizes[count]
                                    size = size_dict['size']
                                    stock = size_dict['availability']
                                    output = output + str(size) + ": " + str(stock) + '\n'
                                    count += 1

                                #If not available, just move on to the next size and check availability
                                else:
                                    count += 1

                            data["username"] = botname
                            data["description"] = "@here"
                            embed["title"] = flag + " " + SKU + " STOCK LATEST UPDATE"
                            embed["description"] = output
                            data["embeds"].append(embed)

                            requests.post(dc_url, data=json.dumps(data), headers={"Content-Type": "application/json"})

                            #The latest scrape take over the previous statuses etc
                            json_first = json_dict
                            status = sta_new

                        #'IN_STOCK' -> 'NOT_AVAILABLE'
                        elif sta_new == 'NOT_AVAILABLE':
                            data["username"] = botname
                            embed["title"] = flag + " " + SKU + " STOCK RAN OUT"
                            embed["description"] = "No more Stock..."
                            data["embeds"].append(embed)

                            requests.post(dc_url, data=json.dumps(data), headers={"Content-Type": "application/json"})

                            #The latest scrape take over the previous statuses etc
                            json_first = json_dict
                            status = sta_new

                    #previous status was 'NOT_AVAILABLE'
                    elif status == "NOT_AVAILABLE":
                        sta_new = json_dict['availability_status']
                        #'NOT_AVAILABLE' -> 'PREVIEW'
                        if sta_new == 'PREVIEW':
                            data["username"] = botname
                            embed["title"] = flag + " " + SKU + " went back to PREVIEW"
                            embed["description"] = "Awaiting further changes"
                            data["embeds"].append(embed)

                            requests.post(dc_url, data=json.dumps(data), headers={"Content-Type": "application/json"})

                            #The latest scrape take over the previous statuses etc
                            json_first = json_dict
                            status = sta_new

                        #'NOT_AVAILABLE' -> 'IN_STOCK'
                        elif sta_new == 'IN_STOCK':
                            size_run = len(json_dict['variation_list'])
                            all_sizes = json_dict['variation_list']
                            count = 0
                            output = ""

                            while count < size_run:
                                if (all_sizes[count])['availability'] != 0:
                                    size_dict = all_sizes[count]
                                    size = size_dict['size']
                                    stock = size_dict['availability']
                                    output = output + str(size) + ": " + str(stock) + '\n'
                                    count += 1

                                #If not available, just move on to the next size and check availability
                                else:
                                    count += 1

                            data["username"] = botname
                            data["description"] = "@here"
                            embed["title"] = flag + " " + SKU + " STOCK RERELEASED!!!"
                            embed["description"] = output
                            data["embeds"].append(embed)

                            requests.post(dc_url, data=json.dumps(data), headers={"Content-Type": "application/json"})

                            #The latest scrape take over the previous statuses etc
                            json_first = json_dict
                            status = sta_new

                        #'NOT_AVAILABLE' -> 'NOT_AVAILABLE'
                        elif sta_new == 'NOT_AVAILABLE':
                            data["username"] = botname
                            embed["title"] = flag + " " + SKU + " NOT AVAILABLE BUT CHANGES DETECTED"
                            embed["description"] = "The JSON output is below to observe the changes\n" + json_first
                            data["embeds"].append(embed)

                            requests.post(dc_url, data=json.dumps(data), headers={"Content-Type": "application/json"})

                            #The latest scrape take over the previous statuses etc
                            json_first = json_dict
                            status = sta_new

            #This portion will update on command prompt to inform user that the program is running and not crashed -> vary the check_duration (seconds) to change the duration of updates
            elif elapsed_time >= (run_count * check_duration):
                cur_time = int(time.time())
                left = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(cur_time))

                right = "Program is running"
                print(str(left) + ": " + right)

                #Reset the start_time to make the loop run infinitely
                if run_count == 10:
                    run_count = 0
                    start_time = time.time()

                run_count += 1

        else:
            data["username"] = botname
            embed["title"] = flag + " " + SKU + " Stock Check Report Ended"
            data["embeds"].append(embed)
            requests.post(dc_url, data=json.dumps(data), headers={"Content-Type": "application/json"})
            break

    except ValueError:
        #cur_time = int(time.time())
        if error_count == 0:
            first_time = int(time.time())
            left = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(first_time))

            right = "Page is being blocked. Will resume when page is not blocked"
            print(str(left + ": " + right))
            error_count += 1

        else:
            next_time = int(time.time())

            if next_time - first_time > check_duration:
                first_time = next_time
                left = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(first_time))
                right = "Page is being blocked. Will resume when page is not blocked"
                print(str(left + ": " + right))

    #Handling ConnectionError, TimeoutError where it will print out the error and keep waiting for 1 minute till making the next request
    except (ConnectionError, TimeoutError) as error:
        left =  time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(int(time.time())))
        right = "Will retry in 1 minute"
        print(left + ": " + right)
        time.sleep(60)

    #KeyboardInterrupt (ctrl C) will stop program and send another webhook to say the program has been stopped
    except KeyboardInterrupt:
        data = {}
        data["embeds"] = []
        embed = {}

        data["username"] = botname
        embed["title"] = "The program has been shut down"
        data["embeds"].append(embed)

        requests.post(dc_url, data=json.dumps(data), headers={"Content-Type": "application/json"})
        break


### Code able to monitor the website to observe the sizes of the sneaker model the user wants to search for. Will notify the user via telegram webhook on the exact stock count right after

### Capable of notifying users when there are changes in the number of sizes available i.e. replenishment of sneakers